In [1]:
import geopandas as gpd
from shapely.geometry import Polygon
from sklearn.cluster import HDBSCAN
import numpy as np

### Input

In [3]:
in_path = r'inputs/' # dummy folder for geodata layers

# geodata layers
bldgs = 'bldgs.shp'
claims = 'claims.shp'
streams = 'streams.shp'
basins = 'basins.shp'
shigs = 'shigs.shp'
roads = 'roads.shp'

# fields in layers
claim_id_field = 'claim_id'
shig_road_field = 'road_id'
road_id_field = 'road_id'
bldg_id_field = 'bldg_id'
shig_id_field = 'shig_id'

### Read data and intersect buildings with claims

In [5]:
bldgs_gdf = gpd.read_file(in_path+bldgs)
claims_gdf = gpd.read_file(in_path+claims)

In [6]:
# convert building geometry to centroid and spatial join with claims
bldgs_gdf['geometry'] = bldgs_gdf.geometry.centroid
bldgs_gdf = bldgs_gdf.sjoin(claims_gdf)

### Compute mean and std building distance to stream by claim

In [8]:
# compute building distance from streams
streams_gdf = gpd.read_file(in_path+streams)
bldgs_gdf['Distance from stream'] = bldgs_gdf.geometry.distance(streams_gdf.geometry.union_all())

In [9]:
# comput mean and std distance from streams by claim and filter claims with less than 10 buildings
stream_claims = bldgs_gdf.groupby(claim_id_field).agg({'Distance from stream':['count', 'mean', 'std']})
stream_claims.columns = stream_claims.columns.droplevel(0)
stream_claims = stream_claims.reset_index()
stream_claims = stream_claims[stream_claims['count']>=10]

### Compute claim-basin/stream similarity

In [11]:
# create fishnet for claims
minX, minY, maxX, maxY = claims_gdf.total_bounds
square_size = 10
geom_array = []
x, y = (minX, minY)
while y <= maxY:
    while x <= maxX:
        geom = Polygon([(x, y), (x, y+square_size), (x+square_size, y+square_size), (x+square_size, y), (x, y)])
        geom_array.append(geom)
        x += square_size
    x = minX
    y += square_size
    
claims_fish = gpd.GeoDataFrame(geom_array, columns=['geometry']).set_crs(claims_gdf.crs)

In [12]:
# keep only fishnet cells that intersect claim boundaries
claims_gdf['geometry'] = claims_gdf.geometry.boundary
claims_fish = claims_fish.sjoin(claims_gdf)

In [13]:
# union basins and streams into a single geometry
basins_gdf = gpd.read_file(in_path+basins)
basins_gdf['geometry'] = basins_gdf.geometry.boundary
basins_gdf = basins_gdf.dissolve()
streams_gdf = streams_gdf.dissolve()
boundaries_gdf = streams_gdf.union(basins_gdf)

In [14]:
# compute cell distance to streams/basin lines and compute average per claim
claims_fish['Boundary alignment'] = claims_fish.geometry.distance(boundaries_gdf.geometry.union_all())
claims_similarity = claims_fish.groupby(claim_id_field)['Boundary alignment'].mean().reset_index()

In [15]:
# merge boundary alignment and stream distances results back into claims data
claims_gdf = claims_gdf.merge(claims_similarity, on=claim_id_field)
claims_gdf = claims_gdf.merge(stream_claims, on=claim_id_field)
claims_gdf.rename(columns={'mean':'Mean stream distance',
                          'std': 'Std. stream distance'}, inplace=True)

### Shig locations

In [17]:
# read Shig and road data
shigs_gdf = gpd.read_file(in_path+shigs)
shigs_gdf = shigs_gdf[~shigs_gdf[shig_road_field].isnull()]
roads_gdf = gpd.read_file(in_path+roads)

### Analysis of clusters

In [19]:
claims_gdf2 = gpd.read_file(in_path+claims)
claims_gdf2 = claims_gdf2[claims_gdf2[claim_id_field].isin(claims_gdf[claim_id_field])]

In [20]:
claims_gdf2 = claims_gdf2[claims_gdf2.intersects(shigs_gdf.geometry.union_all())]

In [21]:
# cluster buildings
claims_with_shigs = claims_gdf[claims_gdf[claim_id_field].isin(claims_gdf2[claim_id_field])][claim_id_field]
hds = HDBSCAN(min_cluster_size=4)
bldgs_gdf['x'] = bldgs_gdf.geometry.x
bldgs_gdf['y'] = bldgs_gdf.geometry.y
X = bldgs_gdf[bldgs_gdf[claim_id_field].isin(claims_with_shigs)][['x', 'y']].to_numpy()
hds.fit(X)
bldgs_gdf.loc[bldgs_gdf[claim_id_field].isin(claims_with_shigs), 'cluster'] = hds.labels_
bldgs_gdf.loc[bldgs_gdf[claim_id_field].isin(claims_with_shigs), 'probabilities'] = hds.probabilities_

In [22]:
# identify each Shig's building, as long as it belongs to a cluster and is no more than 30 m away than the original Shig location
shigs_gdf = shigs_gdf.sjoin_nearest(bldgs_gdf.loc[bldgs_gdf.cluster!=-1, [claim_id_field, 'geometry', bldg_id_field]], distance_col='bldg_dist')
shigs_gdf = shigs_gdf[shigs_gdf['bldg_dist']<=30]

In [23]:
# identify buildings which are Shigs
bldgs_gdf = bldgs_gdf.merge(shigs_gdf[[bldg_id_field, shig_id_field, shig_road_field]], on=bldg_id_field, how='left')
bldgs_gdf['Shig'] = 0
bldgs_gdf.loc[~bldgs_gdf[shig_id_field].isnull(), 'Shig'] = 1

In [24]:
bldgs_gdf['road_distance'] = 1000
for i in bldgs_gdf[bldgs_gdf.Shig==1].cluster.unique(): # iterate through clusters
    # get the access road for shigs in the cluster
    road_idx = bldgs_gdf[(bldgs_gdf.cluster==i) & (bldgs_gdf.Shig==1)][road_id_field].to_numpy()
    road = roads_gdf[roads_gdf[road_id_field].isin(road_idx)]
    # compute distance of all buildings in the cluster to the road and then their percentile values
    bldgs_gdf.loc[bldgs_gdf.cluster==i, 'road_distance'] = bldgs_gdf[bldgs_gdf.cluster==i].geometry.distance(
        road.geometry.union_all())
    bldgs_gdf.loc[bldgs_gdf.cluster==i, 'Shig road distance ranking'] = bldgs_gdf[bldgs_gdf.cluster==i].road_distance.rank(
        pct=True)

C:\Users\Owner\AppData\Local\Temp\ipykernel_18736\2039099664.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[18.66571401  9.57262154 27.20045344  0.12653641]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  bldgs_gdf.loc[bldgs_gdf.cluster==i, 'road_distance'] = bldgs_gdf[bldgs_gdf.cluster==i].geometry.distance(


In [25]:
# compute mean ranking per claim and merge with claims data
claims_shig = bldgs_gdf[bldgs_gdf.Shig==1].groupby(claim_id_field)['Shig road distance ranking'].mean().reset_index()
claims_gdf = claims_gdf.merge(claims_shig, on=claim_id_field, how='left')

### Number of shigs per cluster in clusters with shigs

In [27]:
claims_with_shigs = bldgs_gdf[bldgs_gdf.Shig==1][claim_id_field].unique() # get claims with shigs
# get buildings within these claims
bldgs_in_cws = bldgs_gdf[(bldgs_gdf[claim_id_field].isin(claims_with_shigs)) & (bldgs_gdf.cluster!=-1)]
# count shigs and clusters per claim
shigs_per_cluster = bldgs_in_cws.groupby(claim_id_field).agg({'Shig':'sum', 'cluster':'nunique'}).reset_index()
shigs_per_cluster['Shigs per cluster'] = shigs_per_cluster['Shig'] / shigs_per_cluster['cluster'] # shigs to clusters ratio

In [28]:
# merge with claims data
claims_gdf = claims_gdf.merge(shigs_per_cluster[[claim_id_field, 'Shigs per cluster']], on=claim_id_field, how='left')

In [29]:
claims_gdf

,claim_id,geometry,Boundary alignment,count,Mean stream distance,Std. stream distance,Shig road distance ranking,Shigs per cluster
0,0,"LINESTRING (208206.664 569283.333, 208273.331 ...",17.130759,50,42.604167,22.997209,NaN,NaN
1,3,"LINESTRING (198386.665 569640, 198416.665 5696...",13.965853,44,217.660754,101.157860,NaN,NaN
2,4,"LINESTRING (208756.664 569630, 208743.331 5696...",15.945238,42,42.803086,17.795018,NaN,NaN
3,6,"LINESTRING (208259.998 569743.333, 208279.998 ...",13.602447,20,34.220361,21.551309,NaN,NaN
4,9,"LINESTRING (208796.664 570060, 208836.664 5700...",14.878489,18,43.679043,27.009623,NaN,NaN
...,...,...,...,...,...,...,...,...
140,339,"LINESTRING (207226.664 577806.666, 207229.998 ...",27.851097,48,123.952575,32.248108,0.058824,0.333333
141,340,"LINESTRING (205696.665 577863.332, 205716.665 ...",5.486583,15,78.118206,17.359000,NaN,NaN
142,342,"LINESTRING (203896.665 578139.999, 203896.665 ...",7.700831,66,62.614224,36.947054,0.055556,0.200000
143,344,"LINESTRING (204156.665 578463.332, 204163.331 ...",9.545460,44,53.781632,25.088389,NaN,NaN


In [30]:
# as all other indices show higher similarity when values decrease, use the inverse of the ratio
claims_gdf['Shigs per cluster'] = 1 / claims_gdf['Shigs per cluster']

### EWM-TOPSIS

##### EWM

In [33]:
def norm(X): # normalize values to be share of total
    return X/X.sum()

def EWM(df):
    norm_df = df.apply(norm) # normalize all features
    
    k = -(1/np.log(norm_df.shape[0]))
    
    def entropy(X): # compute entropy value per feature
        return (X*np.log(X)).sum()*k
    
    entropy = norm_df.apply(entropy) # compute entropy for all features
    
    #degree of differentiation
    dod = 1 - entropy

    w = dod/dod.sum() # normalize degree of differentiation = weights
    print(w.sort_values(ascending = False))
    return w

##### TOPSIS

In [35]:
def TOPSIS(df, w):
    def norm(X): # normalize values as the ratio of value to the sum of squares
        return X / np.sqrt((X**2).sum())
    
    norm_matrix = df.apply(norm)
    w_norm_matrix = norm_matrix * w # weight normalized values

    # find the minimal value per feature - since high similarity values are minimal values in this case, this is the 'best' solution
    V_plus = w_norm_matrix.apply(min) 
    V_minus = w_norm_matrix.apply(max) # find the maximal value per feature
    
    S_plus = np.sqrt((w_norm_matrix - V_plus)**2).apply(sum, axis=1) # distance from 'best' solution
    S_minus = np.sqrt((w_norm_matrix - V_minus)**2).apply(sum, axis=1) # distance from 'worst' solution
    
    p_score = S_minus / (S_plus + S_minus) # TOPSIS value
    
    return p_score.reset_index().rename(columns={0:'Score'})

##### EWM_TOPSIS without shigs

In [37]:
df = claims_gdf[[claim_id_field,
                 'Mean stream distance',
                 'Std. stream distance', 
                 'Boundary alignment']].set_index(claim_id_field)
w = EWM(df)
p_score = TOPSIS(df, w)
claims_gdf = claims_gdf.merge(p_score, on=claim_id_field, how='left')

Boundary alignment      0.416419
Mean stream distance    0.299467
Std. stream distance    0.284115
dtype: float64


##### EWM_TOPSIS without shigs, only claims with shigs

In [39]:
df = claims_gdf[~claims_gdf['Shig road distance ranking'].isnull()][[claim_id_field, 
                                                           'Mean stream distance',
                                                           'Std. stream distance', 
                                                           'Boundary alignment']].set_index(claim_id_field)
w2 = EWM(df)
p_score = TOPSIS(df, w2)
claims_gdf = claims_gdf.merge(p_score, on=claim_id_field, how='left')

Boundary alignment      0.588197
Mean stream distance    0.232646
Std. stream distance    0.179157
dtype: float64


##### EWM_TOPSIS with shigs

In [41]:
df = claims_gdf[~claims_gdf['Shig road distance ranking'].isnull()][[claim_id_field, 
                                                           'Mean stream distance',
                                                           'Std. stream distance', 
                                                           'Boundary alignment',
                                                           'Shig road distance ranking',
                                                           'Shigs per cluster']].set_index(claim_id_field)
w3 = EWM(df)
p_score = TOPSIS(df, w3)
claims_gdf = claims_gdf.merge(p_score, on=claim_id_field, how='left')

Shig road distance ranking    0.322545
Boundary alignment            0.262408
Shigs per cluster             0.231333
Mean stream distance          0.103789
Std. stream distance          0.079926
dtype: float64


In [42]:
claims_gdf[['Boundary alignment', 'Mean stream distance', 'Std. stream distance', 'Shig road distance ranking',
           'Shigs per cluster', 'Score', 'Score_x', 'Score_y']].describe()

,Boundary alignment,Mean stream distance,Std. stream distance,Shig road distance ranking,Shigs per cluster,Score,Score_x,Score_y
count,145.000000,145.000000,145.000000,25.000000,25.000000,25.000000,145.000000,25.000000
mean,17.641716,64.473359,29.741358,0.131685,1.856667,0.716107,0.771373,0.712059
std,9.789511,31.020754,13.930971,0.098591,1.151569,0.118091,0.125662,0.208052
min,3.570476,21.672570,9.885717,0.037037,0.500000,0.514431,0.349647,0.192745
25%,10.591421,44.837732,20.902008,0.055556,1.000000,0.622858,0.731381,0.662871
50%,15.530716,56.259627,25.715891,0.106061,1.750000,0.682427,0.810325,0.785797
75%,21.447393,73.834074,34.436259,0.166667,2.000000,0.818959,0.857973,0.837611
max,61.482733,217.660754,101.157860,0.391949,5.000000,0.891870,0.937263,0.932774
